<a href="https://colab.research.google.com/github/ThomasYang0318/MachineLearning/blob/main/%E8%B2%93%E7%8B%97%E8%BE%A8%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我們試著用比較多的照片來做遷移式學習, 看是否有較好的效果。這其實也是一種示範, 看我們如果收集到了一些照片, 怎麼樣整理就能做成訓練資料。

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 讀入函式及貓狗資料

這裡是由 Google 整理的[狗貓辨識資料](https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=MLZKVtE0dSfk)讀入, 我們只會用原本準備好的測試資料, 貓狗各 500 張照片。

In [2]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

從網路讀入一個 `.zip` 檔, 存到我們 Colab 開給我們的 `/content` 資料夾下。

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /content/cats_and_dogs_filtered.zip

--2025-02-26 14:11:38--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.207, 64.233.170.207, 142.251.175.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/content/cats_and_dogs_filtered.zip’

/content/cats_and_d 100%[===================>]  65.43M  20.7MB/s    in 4.0s    

2025-02-26 14:11:42 (16.4 MB/s) - ‘/content/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



這裡示範 `.zip` 檔解壓縮, 解壓縮一樣放到我們的 `/content` 資料夾中。

In [4]:
import os
import zipfile

local_zip = '/content/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

這時在 `/content/cats_and_dogs_filtered/validation/cats` 下有 500 張貓照片, 所以我們把這個路徑記下來叫 `cats_dir`。同理我們也把狗照片路徑記下來, 叫 `dogs_dir`。

In [5]:
base_dir = '/content/cats_and_dogs_filtered/validation'

In [6]:
cats_dir = base_dir + '/cats'
dogs_dir = base_dir + '/dogs'

貓狗照片的檔案名稱, 分別放入 `cat_fnames` 和 `dog_fnames` 兩個串列中。

In [7]:
cat_fnames = os.listdir(cats_dir)
dog_fnames = os.listdir(dogs_dir)

### 2. 準備好輸入輸出

現在 `data` 會放入我們轉成 `array` 的照片, 而 `target` 會是答案: 0 是貓, 狗是 1。

In [8]:
data = []
target = []

for cat in cat_fnames:
    img = load_img(cats_dir + '/' + cat, target_size = (224,224))
    x = np.array(img)
    data.append(x)
    target.append(0)
for dog in dog_fnames:
    img = load_img(dogs_dir + '/' + dog, target_size = (224,224))
    x = np.array(img)
    data.append(x)
    target.append(1)

In [9]:
data = np.array(data)
target = np.array(target)

看看 `data` 的 `shape`, 會發現有 1,000 張 224x224x3 的照片。

In [10]:
data.shape

(1000, 224, 224, 3)

`target` 自然是有 1,000 個正確答案 (貓或狗)。

In [11]:
target.shape

(1000,)

### 4. CNN沒有預處理


In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

# 1. 載入數據 (假設數據已經準備好)
X = data  # 形狀: (num_samples, 224, 224, 3)
y = target  # 形狀: (num_samples,)

# 轉換標籤為 One-Hot 編碼
y = tf.keras.utils.to_categorical(y, num_classes=2)

# 2. 定義 10-Fold 交叉驗證
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 儲存每次結果
results = []

# 3. 交叉驗證循環
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"Training on fold {fold + 1}/10...")

    # 劃分訓練集和驗證集
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # 4. 定義 CNN 模型
    model = Sequential()
    model.add(Conv2D(filters=8, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(224,224,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=16, kernel_size=(3,3), padding='Same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    # 5. 編譯模型
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    # 6. 訓練模型
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_data=(X_val, y_val))

    # 7. 評估模型
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_val, axis=1)

    # 計算指標
    acc = accuracy_score(y_true_classes, y_pred_classes)
    loss = history.history['val_loss'][-1]  # 获取最后一个 epoch 的验证损失
    tn, fp, fn, tp = confusion_matrix(y_true_classes, y_pred_classes).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    auc = roc_auc_score(y_true_classes, y_pred[:, 1]) # 计算AUC，使用预测概率

    # 將结果儲存到列表中
    results.append([fold + 1, acc, loss, sensitivity, specificity, precision, recall, f1, auc])

# 8. 創建 DataFrame 並顯示结果
df_results = pd.DataFrame(results, columns=['Fold', 'Accuracy', 'Loss', 'Sensitivity', 'Specificity', 'Precision', 'Recall', 'F1-score', 'AUC'])
print("\n10-Fold Cross-Validation Results:")
display(df_results)

# 9. 計算平均值和標準差
avg_results = df_results.mean(numeric_only=True)  # 計算数值列的平均值
std_results = df_results.std(numeric_only=True)  # 計算数值列的標準差

# 10. 顯示平均值和標準差
print("\nAverage Results:")
print(avg_results)
print("\nStandard Deviation:")
print(std_results)

Training on fold 1/10...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Training on fold 2/10...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Training on fold 3/10...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step
Training on fold 4/10...


...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Fold']
  ys = series['Specificity']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_results.sort_values('Fold', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Fold')
_ = plt.ylabel('Specificity')

from matplotlib import pyplot as plt
df_results['Fold'].plot(kind='line', figsize=(8, 4), title='Fold')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_results['Accuracy'].plot(kind='line', figsize=(8, 4), title='Accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_results['Loss'].plot(kind='line', figsize=(8, 4), title='Loss')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_results['Sensitivity'].plot(kind='line', figsize=(8, 4), title='Sensitivity')
plt.gca().spines[['top', 'right']].set_visible(False)

### 5. Step 02: 訓練

In [ ]:
!pip install gradio

     |████████████████████████████████| 2.1 MB 9.7 MB/s 
     |████████████████████████████████| 1.9 MB 55.1 MB/s 
     |████████████████████████████████| 206 kB 44.7 MB/s 
     |████████████████████████████████| 961 kB 69.6 MB/s 
     |████████████████████████████████| 3.2 MB 52.0 MB/s 
     |████████████████████████████████| 63 kB 2.9 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4709 sha256=f45d59a509f070a73332ae7d96e939338e3c603a302f543938a772511f4b286b
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3372 sha256=59ee6056a161976b0a5f557c164f83f365dfc9a3f5ea59874b53ac08473cdf24
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


In [ ]:
import gradio as gr

In [ ]:
labels = ['貓', '狗']

In [ ]:
def classify_image(inp):
  inp = inp.reshape((-1, 224, 224, 3))
  inp = preprocess_input(inp)
  p = model.predict(inp).flatten()[0]
  return {'貓': float(1-p), '狗': float(p)}

In [ ]:
image = gr.inputs.Image(shape=(224, 224), label="狗或貓的照片")
label = gr.outputs.Label(label="AI辨識結果")

In [ ]:
gr.Interface(fn=classify_image, inputs=image, outputs=label,
             title="AI 狗貓辨識機",
             description="請輸入一張狗或貓的照片, 看我是否分得出來!"
             ).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://24744.gradio.app
Interface loading below...
